In [59]:
import pandas as pd
import os
import sklearn as sk
from sklearn.linear_model import MultiTaskLasso, MultiTaskElasticNet, ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import sklearn.metrics as skmet
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

In [8]:
os.listdir("inside_airbnb")

['.DS_Store',
 'calendar.csv',
 'listings.csv',
 'listings_summ.csv',
 'neighbourhoods.csv',
 'neighbourhoods.geo.json',
 'neighbourhoods.geojson',
 'neighbourhoods.json',
 'reviews.csv',
 'reviews_summ.csv',
 'stations.geojson']

In [9]:
listings = pd.read_csv('inside_airbnb/listings.csv')
list_summ = pd.read_csv('inside_airbnb/listings_summ.csv')
neighborhoods = pd.read_csv('inside_airbnb/neighbourhoods.csv')
reviews = pd.read_csv('inside_airbnb/reviews.csv')
reviews_summ = pd.read_csv('inside_airbnb/reviews_summ.csv')
calendar = pd.read_csv('inside_airbnb/calendar.csv')

In [61]:
stopWords = set(stopwords.words('english'))

In [11]:
reviews['comment_length'] = reviews['comments'].str.len()

In [12]:
listings

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,11204286,https://www.airbnb.com/rooms/11204286,20160706203047,2016-07-07,Family friendly/California king,"Aquatica Waterpark, Sleep train Amphitheater, ...","Walking to Aquatica Waterpark, Sleep train Amp...","Aquatica Waterpark, Sleep train Amphitheater, ...",none,NaN,...,10.0,f,NaN,"SAN DIEGO, SAN DIEGO TOURISM MARKETING DISTRIC...",f,moderate,f,f,2,4.57
1,7972006,https://www.airbnb.com/rooms/7972006,20160706203047,2016-07-07,Welcome to Sunset Suite,Your spacious room awaiting is with a Queen Si...,NaN,Your spacious room awaiting is with a Queen Si...,none,Getting around is easy. Very close to Eastlake...,...,10.0,f,NaN,NaN,f,strict,f,f,1,0.76
2,7727710,https://www.airbnb.com/rooms/7727710,20160706203047,2016-07-07,San Diego/Eastlake. Gated community,"This is an immaculate 3 bedroom, 2 1/2 bath co...",NaN,"This is an immaculate 3 bedroom, 2 1/2 bath co...",none,NaN,...,8.0,f,NaN,NaN,f,flexible,f,f,1,0.09
3,13124681,https://www.airbnb.com/rooms/13124681,20160706203047,2016-07-07,Townhome in Eastlake,This 2 Story TownHome is close to Otay Ranch ...,"My place is good for couples, business travele...",This 2 Story TownHome is close to Otay Ranch ...,none,"Located in eastern Chula Vista, Otay Ranch is ...",...,10.0,f,NaN,NaN,f,moderate,f,f,1,3.00
4,3469225,https://www.airbnb.com/rooms/3469225,20160706203047,2016-07-07,Bedroom suite in Large new home,Hello; we are offering a private secluded bedr...,"Beautiful, quiet award-winning suburban neighb...",Hello; we are offering a private secluded bedr...,none,"The quiet serenity; near Park and lakes, beaut...",...,NaN,f,NaN,NaN,f,strict,f,f,2,NaN
5,877473,https://www.airbnb.com/rooms/877473,20160706203047,2016-07-07,Private Bedroom in Newer Home,"Private Bedroom with full size bed, + futon, p...","Private bedroom with full size Bed, in large n...","Private Bedroom with full size bed, + futon, p...",none,"It is quiet, serene, tranquil, suburban; yet c...",...,10.0,f,NaN,NaN,f,strict,f,f,2,0.63
6,3124507,https://www.airbnb.com/rooms/3124507,20160706203047,2016-07-07,"Great view to mountains,lake,trails","In this beautiful home, located near the Olymp...",The location and view of lake and mountain,"In this beautiful home, located near the Olymp...",none,Like countryside and also near from downtown S...,...,10.0,f,NaN,NaN,f,strict,f,f,3,0.31
7,3432507,https://www.airbnb.com/rooms/3432507,20160706203047,2016-07-07,2 bedrooms with shared bath,In a great house with view on Otay Lakes and ...,"the proximity to lake ,trail and Olympic train...",In a great house with view on Otay Lakes and ...,none,Freedom in nature,...,10.0,f,NaN,NaN,f,strict,f,f,3,0.04
8,3249729,https://www.airbnb.com/rooms/3249729,20160706203047,2016-07-07,Near to Olympic Training center,Beautiful home in the Eastlake community with ...,Location on the border of the town with nice t...,Beautiful home in the Eastlake community with ...,none,"open , quiet, country feeling",...,10.0,f,NaN,NaN,f,strict,f,f,3,0.08
9,11756336,https://www.airbnb.com/rooms/11756336,20160706203047,2016-07-07,Chula Vista Golf Course House,"Feel right at home in our spacious 5BR, 3.5BA ...","Feel right at home in our spacious 5BR, 3.5BA ...","Feel right at home in our spacious 5BR, 3.5BA ...",none,Our home is surrounded by natural beauty. Visi...,...,8.0,f,NaN,NaN,f,strict,f,f,5,0.59


In [13]:
#Confirmation that all ids are unique 
listings['id'].nunique()

6608

In [20]:
#Using 11204286 as an example. Number of reviews is 7.
listings[listings['id'] == 11204286]['number_of_reviews'] 

0    7
Name: number_of_reviews, dtype: int64

In [25]:
#taking the number of reviews in the reviews file for id 11204286 (check if they are equal)
len(reviews[reviews['listing_id'] == 11204286])

7

In [35]:
sum(reviews[reviews['listing_id'] == 11204286]['comment_length'])

1954.0

In [24]:
#Checking if there are no NaN in the review count (the length is the same as the original length)
len(listings['number_of_reviews'].dropna())

6608

In [36]:
#Average review length is sum(length_reviews)/count(reviews)
listings['average_review_length'] = 0

for idx in listings['id'].index:
    w = listings['id'][idx]
    a = listings[listings['id'] == w]['number_of_reviews'].item()
    
    if a == 0:
        continue
        
    b = sum(reviews[reviews['listing_id'] == w]['comment_length'])
    listings['average_review_length'][idx] = float(b)/float(a)
    

C:\Users\sanka\Anaconda2\lib\site-packages\ipykernel\__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [43]:
#Computing a weight based on reviewer's history
L = []
for w in reviews['reviewer_id'].unique():
    a = len(reviews[reviews['reviewer_id'] == w])
    if a > 1:
        L.append((w, a))
    
L

[(61280862, 3),
 (3628691, 3),
 (38839815, 2),
 (2804750, 2),
 (68072119, 3),
 (15746017, 2),
 (57166342, 2),
 (18004048, 2),
 (56405589, 2),
 (9913594, 2),
 (43625266, 2),
 (33480788, 2),
 (1009467, 2),
 (23700601, 2),
 (46179252, 6),
 (27714399, 2),
 (13430797, 2),
 (7275138, 2),
 (8354170, 3),
 (11101380, 2),
 (14368568, 2),
 (8479607, 2),
 (17602325, 2),
 (9650481, 2),
 (44670009, 2),
 (25012727, 3),
 (19532074, 2),
 (66281042, 3),
 (35734676, 2),
 (10175086, 4),
 (65940377, 2),
 (50155059, 2),
 (44751786, 2),
 (9037735, 2),
 (13223821, 2),
 (18383086, 2),
 (3377035, 6),
 (25774740, 3),
 (41525019, 2),
 (59357545, 2),
 (71495097, 2),
 (75206075, 2),
 (2824843, 6),
 (51691026, 3),
 (67221522, 2),
 (18093312, 3),
 (26919407, 2),
 (324652, 3),
 (26127554, 2),
 (27748750, 3),
 (2592550, 2),
 (5634438, 3),
 (2733694, 2),
 (11291312, 2),
 (27564373, 3),
 (283119, 2),
 (46364, 3),
 (46077279, 2),
 (14410864, 7),
 (59453156, 2),
 (46183205, 9),
 (45361366, 2),
 (13387816, 5),
 (31159455, 2

In [50]:
#Converting list to dictionary 
reviewerid_count_dict = {}

for w in L:
    reviewerid_count_dict[w[0]] = w[1]

In [53]:
#Creating the reviewer_weight column 
reviews['reviewer_weight'] = 1

for w in reviews.index:
    a = reviews['reviewer_id'][w].item()
    
    if a in reviewerid_count_dict.keys():
        b = reviewerid_count_dict[a]
        reviews['reviewer_weight'][w] = b

C:\Users\sanka\Anaconda2\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [56]:
#Checking if it went well (and it did)
reviews['reviewer_weight'].unique()

array([ 1,  3,  2,  6,  4,  7,  9,  5, 17, 10, 15, 19,  8, 27, 11, 14, 12,
       26], dtype=int64)

In [62]:
reviews['comments_stop_words_ratio'] = 0

for w in reviews.index:
    count = 0
    
    for word in reviews['comments'][w].split(' '):
        if word in stopWords:
            count = count + 1
    
    reviews['comments_stop_words_ratio'][w] = float(count)/(reviews['comment_word_count'][w])     

C:\Users\sanka\Anaconda2\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


AttributeError: 'float' object has no attribute 'split'

In [57]:
reviews

,listing_id,id,date,reviewer_id,reviewer_name,comments,comment_length,comment_word_count,reviewer_weight
0,4796400,32687022,2015-05-20,29916246,Bill,"Jeff, we cannot thank you enough for your hosp...",657.0,116,1
1,4796400,33062004,2015-05-25,3547281,Christina,"The location was amazing, we were able to chea...",265.0,42,1
2,4796400,41311067,2015-08-07,16458913,Naif,I would like to thank Jeff for his great hospi...,220.0,43,1
3,4796400,57640415,2015-12-28,3199045,Matt,Jeff was an amazing host! We rented all three ...,722.0,123,1
4,4796400,58316432,2016-01-02,47834158,Sally,Jeff was the consummate host. He was quick to...,677.0,117,1
5,6716557,74703862,2016-05-16,67892303,Mattie,"The location was excellent, literally ste...",313.0,62,1
6,6716557,77373202,2016-05-31,58474306,Gretchen,Wonderful beach house. So close to the beach. ...,101.0,22,1
7,12493871,73952032,2016-05-11,57841638,Stefan,perfect place for me and my family. would love...,59.0,12,1
8,12493871,80329505,2016-06-17,61280862,Taryn,Ryan and his staff were amazing!!!!! So incred...,139.0,22,3
9,12493871,81695415,2016-06-24,72024589,SaraJo,My communication with Ryan before our stay was...,1685.0,338,1


In [67]:
reviews[reviews['reviewer_id'] == 8222312]['listing_id'].unique()

array([2464739, 1919622, 2142779,  519919], dtype=int64)

In [68]:
reviewer_listing_pairs = reviews[['listing_id', 'reviewer_id', 'reviewer_name']]

In [69]:
reviewer_listing_pairs

,listing_id,reviewer_id,reviewer_name
0,4796400,29916246,Bill
1,4796400,3547281,Christina
2,4796400,16458913,Naif
3,4796400,3199045,Matt
4,4796400,47834158,Sally
5,6716557,67892303,Mattie
6,6716557,58474306,Gretchen
7,12493871,57841638,Stefan
8,12493871,61280862,Taryn
9,12493871,72024589,SaraJo


In [75]:
reviewer_listing_pairs.groupby(['listing_id']).count().head(10)

,reviewer_id,reviewer_name
listing_id,,
6,44,44
9553,2,2
14668,165,165
37149,3,3
38245,79,79
39516,241,241
45429,6,6
54001,87,87
62274,299,299


In [81]:
len(reviewer_listing_pairs.groupby(['listing_id']).count())

4590

In [76]:
reviewer_listing_pairs.groupby(['reviewer_id']).count().head(10)

,listing_id,reviewer_name
reviewer_id,,
248,1,1
415,1,1
899,1,1
1467,1,1
1767,1,1
2288,1,1
2363,1,1
2425,1,1
2675,1,1


In [80]:
len(reviewer_listing_pairs.groupby(['reviewer_id']).count())

85263

In [79]:
reviewer_listing_pairs.groupby(['listing_id', 'reviewer_id']).count().head()

reviewer_name
listing_id reviewer_id               
6          415                      1
           3762                     1
           52739                    1
           117966                   1
           127886                   1

In [82]:
len(reviewer_listing_pairs.groupby(['listing_id', 'reviewer_id']).count())

91718

In [78]:
reviews

,listing_id,id,date,reviewer_id,reviewer_name,comments,comment_length,comment_word_count,reviewer_weight,comments_stop_words_ratio
0,4796400,32687022,2015-05-20,29916246,Bill,"Jeff, we cannot thank you enough for your hosp...",657.0,116,1,0
1,4796400,33062004,2015-05-25,3547281,Christina,"The location was amazing, we were able to chea...",265.0,42,1,0
2,4796400,41311067,2015-08-07,16458913,Naif,I would like to thank Jeff for his great hospi...,220.0,43,1,0
3,4796400,57640415,2015-12-28,3199045,Matt,Jeff was an amazing host! We rented all three ...,722.0,123,1,0
4,4796400,58316432,2016-01-02,47834158,Sally,Jeff was the consummate host. He was quick to...,677.0,117,1,0
5,6716557,74703862,2016-05-16,67892303,Mattie,"The location was excellent, literally ste...",313.0,62,1,0
6,6716557,77373202,2016-05-31,58474306,Gretchen,Wonderful beach house. So close to the beach. ...,101.0,22,1,0
7,12493871,73952032,2016-05-11,57841638,Stefan,perfect place for me and my family. would love...,59.0,12,1,0
8,12493871,80329505,2016-06-17,61280862,Taryn,Ryan and his staff were amazing!!!!! So incred...,139.0,22,3,0
9,12493871,81695415,2016-06-24,72024589,SaraJo,My communication with Ryan before our stay was...,1685.0,338,1,0
